<a href="https://colab.research.google.com/github/jaanushma/Twitter_Sentiment_Analysis/blob/main/Copy_of_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle


In [ ]:
!git clone  https://github.com/jaanushma/Twitter_Sentiment_Analysis.git

Cloning into 'Twitter_Sentiment_Analysis'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


# New section

In [2]:
!mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Import the dataset

In [3]:
#using api to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 70% 57.0M/80.9M [00:00<00:00, 192MB/s]
100% 80.9M/80.9M [00:00<00:00, 178MB/s]


In [4]:
#extract the compressed dataset
from zipfile import ZipFile

dataset ='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print('The dataset is extracted')
  # with ZipFile(dataset,'r') as zip:
  #   zip.extractall()
  #   print('The dataset is extracted')

The dataset is extracted


In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#priinting the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
#load the data into dataframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [9]:
#DIMENSION
twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#renaming the dataset
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [12]:
twitter_data.shape
twitter_data.head(5)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
#check missing values
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [14]:
#check the  ditribution of target column
twitter_data['target'].value_counts()


,count
target,
0,800000
4,800000


In [15]:
#conversion of "4" into 1
twitter_data.replace({'target':{4:1}},inplace=True)


In [16]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0  means--> negative tweet
1  means--> positive tweet


*Stemming* is the process of reducing a word to its root word
ex:dancing,jumping, classical dancing=>dance


In [17]:
port_stem=PorterStemmer()

In [18]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not  word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [21]:
twitter_data.head(5)

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [22]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [24]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [25]:
#seperate the data and label
X= twitter_data['stemmed_content'].values
Y= twitter_data['target'].values

In [26]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [27]:
print(Y)

[0 0 0 ... 1 1 1]


In [28]:
#split the data and train +test
X_train,X_test, Y_train,Y_test=train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)

In [29]:
print(X.shape, X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [31]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


using vectorizer
convert the actual data -->numerical data


In [32]:
vectorizer=TfidfVectorizer()
X_train =vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [33]:
print(X_train)

  (0, 458421)	0.27228112326228093
  (0, 372989)	0.3583819096048567
  (0, 194823)	0.5274064910069903
  (0, 116318)	0.3749338694119814
  (0, 247667)	0.4215060595161854
  (0, 464940)	0.4483581441053542
  (1, 169453)	1.0
  (2, 116318)	0.45905705320412793
  (2, 132152)	0.1891875792321668
  (2, 427967)	0.18706371586911208
  (2, 137276)	0.29069581876534506
  (2, 427034)	0.32103058232997417
  (2, 455221)	0.3296073093697297
  (2, 83506)	0.31303964332256906
  (2, 465311)	0.33480686186877023
  (2, 281186)	0.24137202118772233
  (2, 429906)	0.15168139293444055
  (2, 187474)	0.16194060015844067
  (2, 159240)	0.18800868496584808
  (2, 140239)	0.20289994039686138
  (2, 304223)	0.1678648733603879
  (3, 427034)	0.29025988953598614
  (3, 167435)	0.44570129071167713
  (3, 160331)	0.2785087322306358
  (3, 61032)	0.52009780709937
  :	:
  (1279996, 335154)	0.21190629789318696
  (1279996, 455685)	0.27108924963905606
  (1279996, 409874)	0.21996814268672968
  (1279996, 392260)	0.3596073367651944
  (1279996, 250

In [34]:
print(X_test)

  (0, 17977)	0.17180827032540266
  (0, 34872)	0.16236726920078096
  (0, 72943)	0.2678484639307855
  (0, 112919)	0.367081693610678
  (0, 140294)	0.25509664355434203
  (0, 146252)	0.23682534421364126
  (0, 180544)	0.2803743470609849
  (0, 285841)	0.4532311769636366
  (0, 294440)	0.17813419553395948
  (0, 408024)	0.21977439030363208
  (0, 419078)	0.34884649404705365
  (0, 429906)	0.3140986667345934
  (0, 442233)	0.17903569516819148
  (1, 8801)	0.30376315944222493
  (1, 17977)	0.2084958588978494
  (1, 153709)	0.5722666578553444
  (1, 229396)	0.40557814327518943
  (1, 270721)	0.28432607616832933
  (1, 366102)	0.4820488545932507
  (1, 385085)	0.24302725359248564
  (2, 25826)	0.3529227215218366
  (2, 38175)	0.37941511785250515
  (2, 95488)	0.363168936008175
  (2, 192882)	0.5885804190746107
  (2, 270795)	0.2562344190631401
  :	:
  (319994, 465688)	0.27802041016790324
  (319995, 114775)	0.3338149615345875
  (319995, 116391)	0.301928166814025
  (319995, 164141)	0.27692991319244975
  (319995, 224

train the  model
{Logistic Regression}

In [35]:
model=LogisticRegression(max_iter=1000)

In [36]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [37]:
#accuracy score on the trainig data
X_train_prediction= model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train,X_train_prediction)

In [38]:
print('Accuracy score on the training data :',training_data_accuracy)

Accuracy score on the training data : 0.79812734375


In [39]:
X_test_prediction =model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)


In [40]:
print('Accuracy score on the training data :',training_data_accuracy)

Accuracy score on the training data : 0.79812734375


Model accuracy=79%

Saving the trained model

In [41]:
import pickle


In [42]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the saved model for future predictions

In [43]:
#loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))


In [44]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)
if (prediction[0] == 0):
  print('Negative Tweet')
else:
    print('Positive Tweet')

1
[1]
Positive Tweet


In [45]:
X_new = X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)
if (prediction[0] == 0):
  print('Negative Tweet')
else:
    print('Positive Tweet')

0
[0]
Negative Tweet
